# Y.Music

# Konten <a id='back'></a>

* [Pendahuluan](#intro)
* [Tahap 1. Ikhtisar Data](#data_review)
    * [Kesimpulan](#data_review_conclusions)
* [Tahap 2. Pra-pemrosesan data](#data_preprocessing)
    * [2.1 Gaya Penulisan Judul](#header_style)
    * [2.2 Nilai-Nilai yang Hilang](#missing_values)
    * [2.3 Duplikat](#duplicates)
    * [2.4 Kesimpulan](#data_preprocessing_conclusions)
* [Tahap 3. Pengujian Hipotesis](#hypotheses)
    * [3.1 Hipotesis 1: Aktivitas pengguna di kedua kota](#activity)
    * [3.2 Hipotesis 2: Preferensi musik pada hari Senin dan Jumat](#week)
    * [3.3 Hipotesis 3: Preferensi genre di kota Springfield dan Shelbyville](#genre)
* [Temuan](#end)

## Pendahuluan <a id='intro'></a>
Setiap kali melakukan analisis, perlu merumuskan beberapa hipotesis yang perlu diuji lebih lanjut. Terkadang, pengujian yang dilakukan membawa kita untuk menerima hipotesis tersebut. Namun di lain waktu, kita juga perlu menolaknya. Untuk membuat keputusan yang tepat dalam bisnis, kita harus memahami apakah asumsi yang kita buat sudah tepat atau belum.

Dalam proyek ini, akan membandingkan preferensi musik dari pendengar di kota Springfield dan Shelbyville. Lalu akan meninjau data dari Y.Music untuk menguji beberapa hipotesis di bawah ini dan membandingkan perilaku pengguna pada kedua kota.

### Tujuan: 
Menguji tiga hipotesis:
1. Aktivitas pengguna berbeda-beda tergantung pada hari dan kotanya.
2. Pada hari Senin pagi, penduduk Springfield dan Shelbyville mendengarkan genre yang berbeda. Hal ini juga berlaku untuk hari Jumat malam.
3. Pendengar di kota Springfield dan Shelbyville memiliki preferensi yang berbeda. Di Springfield, para pengguna lebih suka musik pop, sementara di Shelbyville musik rap memiliki lebih banyak penggemar.

### Tahapan
Data terkait perilaku pengguna disimpan dalam file `music_project_en.csv`. Tidak ada informasi terkait kualitas data tersebut, karena itu perlu memeriksanya terlebih dahulu sebelum menguji hipotesis.

Pertama mengevaluasi kualitas data dan melihat apakah masalahnya signifikan. Kemudian, selama pra-pemrosesan data, akan dicoba mengatasi masalah yang paling serius.
 
Proyek ini terdiri dari tiga tahap:
 1. Tinjauan data
 2. Pra-pemrosesan data
 3. Pengujian hipotesis

 

## Tahap 1. Tinjauan data <a id='data_review'></a>

In [24]:
# Mengimpor library Pandas
import pandas as pd

In [25]:
# Membaca file dan menyimpannya ke variabel df
try:
    df=pd.read_csv('/datasets/music_project_en.csv')
except:
    df=pd.read_csv('music_project_en.csv')

In [26]:
# Menampilkan 10 baris data pertama dari tabel df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [27]:
# Memperoleh informasi umum tentang data yang tersedia di df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [28]:
# Mengetahui evaluasi data
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


In [29]:
# Mengetahui jumlah data
df.shape

(65079, 7)

In [30]:
# Mengetahui type data dan jumlah kolom
df.dtypes

  userID    object
Track       object
artist      object
genre       object
  City      object
time        object
Day         object
dtype: object

Tabel ini berisi tujuh kolom. Semua kolom memiliki tipe data yang sama, yaitu: `object`.

Berdasarkan dokumentasi:
- `'userID'` — ID pengguna
- `'Track'` — judul lagu
- `'artist'` — nama artis
- `'genre'` — jenis musik
- `'City'` — kota asal pengguna
- `'time'` — waktu saat lagu tersebut diputar
- `'Day'` — hari dalam seminggu

Terdapat masalah dengan gaya penulisan nama kolom:
1. Beberapa nama ditulis dalam huruf besar, beberapa dalam huruf kecil.
2. Beberapa nama menggunakan spasi.
3. Penulisan nama kolom tidak mengikuti aturan baku seperti aturan snake_case.

Kita juga dapat melihat bahwa terdapat jumlah nilai yang berbeda antar kolom. Hal ini menandakan bahwa data yang kita miliki mengandung nilai yang hilang.


### Kesimpulan <a id='data_review_conclusions'></a> 

Setiap baris dalam tabel menyimpan data terkait trek lagu yang diputar. Beberapa kolom menyimpan data yang mendeskripsikan trek itu sendiri: judul lagu, artis, dan genre. Sisanya menyimpan data terkait informasi pengguna: kota asal mereka, waktu mereka memutar trek lagu tersebut.

Jelas bahwa data yang kita miliki cukup untuk menguji hipotesis. Sayangnya, terdapat sejumlah nilai yang hilang.

Untuk melanjutkan analisis, kita perlu melakukan pra-pemrosesan data terlebih dahulu.

## Tahap 2. Pra-pemrosesan data <a id='data_preprocessing'></a>

Dalam data yang dimiliki terdapat permasalahan yang kemungkinan ada, seperti penamaan kolom yang tidak sesuai, duplikasi data, data yang hilang, dan lain-lain. 
Untuk Itu perlu dilakukan pengecekan lebih dalam lagi pada data yang dimiliki. Untuk memastikan hasil analisis nantinya mendukung hipotesis yang sebelumnya sudah dibuat.

### Gaya penulisan judul <a id='header_style'></a>
Akan dilakukan perbaikan pada format judul kolom 


In [31]:
# list yang memuat nama-nama kolom pada tabel df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Ubah nama kolom sesuai dengan aturan gaya penulisan yang baik:
* Jika nama kolom terdiri dari beberapa kata, gunakan snake_case
* Semua karakter harus menggunakan huruf kecil
* Hapus spasi

In [34]:
# Menstandarisasi nama kolom
df=df.rename(columns={'  userID':'user_id','Track':'track','artist':'artist','genre':'genre',
                      '  City  ':'city', 'time':'time','Day':'day'})

In [35]:
# Mengecek perubahan
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Nilai-nilai yang hilang <a id='missing_values'></a>
Pertama, temukan jumlah nilai yang hilang dalam tabel. Untuk melakukannya, gunakan dua metode `Pandas`:

In [36]:
# Menghitung nilai yang hilang pada setiap kolom
df.isna().sum().sort_values(ascending=False)

artist     7567
track      1343
genre      1198
user_id       0
city          0
time          0
day           0
dtype: int64

Dari informasi data diatas terdapat data hilang pada kolom **artist, track, dan genre** sedang pada kolom lainnya tidak terdapat data yang hilang

Tidak semua nilai yang hilang berpengaruh terhadap penelitian nantinya. Untuk nilai yang hilang dalam kolom **`track` dan `artist`** tidak begitu penting, cukup menggantinya dengan penanda yang jelas.

Akan tetapi, nilai yang hilang dalam kolom `'genre'` dapat memengaruhi perbandingan preferensi musik di kota Springfield dan Shelbyville. 

Oleh karena itu, saat ini yang perlu dilakukan:
* Mengisi nilai yang hilang pada kolom yang teridentifikasi dengan **penanda** (dalam hal ini akan diganti dengan **unknown**)
* Mengevaluasi seberapa besar pengaruh nilai yang hilang terhadap perhitungan

In [38]:
# Melakukan perubahan untuk nilai yang hilang dengan penanda 'unknown'
columns_to_replace = ['artist','genre','track']
for columns in columns_to_replace:
    df[columns] = df[columns].fillna('unknown')

Pengecekan kembali apakah masih ada untuk nilai yang hilang .

In [40]:
# Melakukan cek perubahan untuk nilai yang hilang
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplikat Data<a id='duplicates'></a>


In [41]:
# Melakukan pengecekan dan menghitung duplikasi data eksplisit
df.duplicated().sum()

3826

Dari informasi data diatas terdapat duplikasi data sebanyak **3826** baris data

In [42]:
# Melakukan penghapusan duplikasi data eksplisit
df=df.drop_duplicates().reset_index()

Hitung duplikat eksplisit sekali lagi untuk memastikan bahwa tidak ada lagi duplikasi data.

In [44]:
# Memeriksa duplikat
df.duplicated().sum()

0

Sekarang, hapus duplikat implisit di kolom `genre`. Sebagai contoh, penulisan suatu nama genre dengan cara yang berbeda merupakan contoh dari duplikat implisit. Kesalahan seperti ini juga akan memengaruhi hasil analisismu.

Tampilkan list yang memuat nama unik genre, lalu urutkan list tersebut berdasarkan abjad. Untuk melakukannya:
* Ambil kolom DataFrame yang diinginkan
* Terapkan metode pengurutan pada kolom tersebut\n",
* Untuk kolom yang telah diurutkan, panggil metode yang akan menghasilkan semua nilai unik kolom

In [45]:
# Menampilkan nama genre yang unik
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

Pada list diatas yang telah ditampilkan terdapat duplikat implisit dari genre `hiphop`. Duplikat tersebut bisa berupa nama yang ditulis secara tidak tepat atau nama alternatif dari genre yang sama.

duplikat implisit tersebut yaitu:
* `hip`
* `hop`
* `hip-hop`

Perlu dilakukan perbaikan untuk nilai duplikasi implisit tersebut. Untuk nilai tersebut akan dirubah menjadi 1 nilai yaitu **`hip-hop`**.

In [46]:
# Membuat Fungsi untuk mengganti nilai duplikat implisit
def replace_wrong_genres(wrong_values, correct_value):
    for wrong_value in wrong_values: 
        df['genre'] = df['genre'].replace(wrong_value, correct_value)

In [47]:
# Menjalankan fungsi yang sudah dibuat
wrong_genres = ['hip','hop','hip-hop']# list yang memuat nama dengan pengejaan yang salah
correct_genre = 'hiphop' # nama yang benar
replace_wrong_genres(wrong_genres, correct_genre) # pemanggilan fungsi

Pastikan kembali pada data bahwa nilai yang terduplikasi telah digantikan dengan menampilkan list nilai unik dari kolom `'genre'` kembali:

In [48]:
# Memeriksa duplikat implisit
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

### Kesimpulan <a id='data_preprocessing_conclusions'></a>
Sampai saat ini sudah mendeteksi tiga masalah dalam data yang ada:

1. Gaya penulisan judul yang salah
2. Nilai yang hilang (digantikan dengan penanda yaitu **unknown**)
3. Duplikat eksplisit dan implisit

Sekarang, nama-nama kolom telah dibersihkan untuk mempermudah pemrosesan tabel.
Semua nilai yang hilang pun telah diganti dengan `'unknown'`. Meski demikian, harus dilihat kembali apakah nilai yang hilang dalam kolom `'genre'` akan memengaruhi perhitungan kita.

Ketiadaan duplikat akan membuat hasil yang kita dapat menjadi lebih tepat dan lebih mudah dipahami.

## Tahap 3. Pengujian hipotesis <a id='hypotheses'></a>

### Hipotesis 1: membandingkan perilaku pengguna di dua kota <a id='activity'></a>

Menurut hipotesis pertama, pengguna dari kota Springfield dan Shelbyville memiliki perbedaan perilaku dalam mendengarkan musik. Pengujian ini menggunakan data yang diambil dari tiga hari dalam seminggu: Senin, Rabu, dan Jumat.

Untuk itu diperlukan:

1. Membagi pengguna ke dalam beberapa kelompok berdasarkan kota.
2. Membandingkan berapa banyak trek lagu yang dimainkan oleh setiap kelompok pada hari Senin, Rabu, dan Jumat.

In [51]:
# Menghitung trek lagu yang diputar di setiap kota
count_per_kota = df.groupby('city')['track'].count()
count_per_kota

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

Dari informasi yang didapatkan diketahui bahwa pengguna dari kota **Springfield** memutar lebih banyak trek lagu daripada pengguna dari kota **Shelbyville**. Akan tetapi, hal ini tidak serta-merta mengisyaratkan bahwa warga Springfield lebih sering mendengarkan musik. Kota ini memang lebih besar, dan terdapat lebih banyak pengguna. Jadi, ini merupakan hal yang wajar.

Sekarang, mengelompokkan data berdasarkan hari dan temukan jumlah trek lagu yang diputar pada hari Senin, Rabu, dan Jumat.

In [52]:
# Menghitung trek lagu yang diputar pada masing-masing hari
count_per_day= df.groupby('day')['track'].count()
count_per_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

Dapat diketahui bahwa Rabu adalah hari yang paling **tenang** secara keseluruhan. Namun jika kita mempertimbangkan kedua kota secara terpisah, kita mungkin akan mendapatkan kesimpulan yang berbeda.

Selanjutnya kita akan membuat fungsi. Fungsi ini nantinya berguna untuk mengetahui jumlah Track yang dimainkan dikedua kota pada masing-masing hari (senin, rabu, dan jumat)

In [55]:
# Membuat fungsi number_tracks(), untuk mengetahui jumlah lagu yang dimainkan dikota apa dan pada hari yang ada dalam data.
def number_tracks(day, city):
    track_list = df[ (df['day']==day) & (df['city']==city) ]
    track_list_counts = track_list['user_id'].count() #menghitung user id
    return (track_list_counts)

Setelah fungsi dibuat, saatnya mengaplikasikannya dikedua kota dan pada masing-masing hari (Senin, Rabu, dan Jumat).

In [56]:
# Mengetahui jumlah lagu yang diputar di Springfield pada hari Senin
spr_mon=number_tracks('Monday', 'Springfield')
spr_mon

15740

In [57]:
# Mengetahui jumlah lagu yang diputar di Shelbyville pada hari Senin
shel_mon=number_tracks('Monday', 'Shelbyville')
shel_mon

5614

In [58]:
# Mengetahui jumlah lagu yang diputar di Springfield pada hari Rabu
spr_wed=number_tracks('Wednesday', 'Springfield')
spr_wed

11056

In [59]:
# Mengetahui jumlah lagu yang diputar di Shelbyville pada hari Rabu
shel_wed=number_tracks('Wednesday', 'Shelbyville')
shel_wed

7003

In [60]:
# Mengetahui jumlah lagu yang diputar di Springfield pada hari Jumat
spr_fri=number_tracks('Friday', 'Springfield')
spr_fri

15945

In [61]:
# Mengetahui jumlah lagu yang diputar di Shelbyville pada hari Jumat
shel_fri=number_tracks('Friday', 'Shelbyville')
shel_fri

5895

Selanjutnya dari hasil diatas akan dibuatkan kedalam 1 tabel baru yang menampung ke-6 nilai tersebut untuk dapat kita analisa kembali berikut detail tabel baru tersebut:
* Untuk nama kolomnya adalah: `['city', 'monday', 'wednesday', 'friday']`
* Untuk Datanya adalah hasil dari `number_tracks()`

In [62]:
# Membuat tabel baru data_music yang memuat hasil fungsi number_tracks

data = [
    ['Springfield', spr_mon, spr_wed, spr_fri],
    ['Shelbyville', shel_mon, shel_wed, shel_fri]
]

entries = ['city', 'monday', 'wednesday', 'friday']

data_music = pd.DataFrame(data=data, columns=entries)

data_music

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Kesimpulan**

Dari data yang didapatkan ternyata berhasil mengungkapkan beberapa perbedaan perilaku pengguna:

1. Di kota Springfield, jumlah trek lagu yang diputar mencapai puncaknya pada hari Senin dan Jumat, sedangkan pada hari Rabu terjadi penurunan aktivitas.

2. Di kota Shelbyville, sebaliknya, pengguna lebih banyak mendengarkan musik pada hari Rabu. Aktivitas pengguna pada hari Senin dan Jumat lebih sedikit. 

Dengan demikian, dapat disimpulkan bahwa hipotesis pertama tampaknya benar.

### Hipotesis 2: musik di awal dan akhir minggu <a id='week'></a>

Menurut hipotesis kedua, pada hari Senin pagi dan Jumat malam, warga Springfield mendengarkan genre musik yang berbeda dari yang dinikmati warga Shelbyville.

Diperlukan membuat  tabel yang memuat data khusus kedua kota, untuk selanjutnya dilakukan pengecekan untuk menguji hipotesis kedua:
* Untuk Springfield — `spr_general`
* Untuk Shelbyville — `shel_general`

In [63]:
# Membuat tabel spr_general dari baris df, yang nilai dari kolom 'city'-nya adalah 'Springfield'
spr_general= df.loc[df['city']=='Springfield']
spr_general

,index,user_id,track,artist,genre,city,time,day
1,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
4,4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
6,6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
...,...,...,...,...,...,...,...,...
61247,65073,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Springfield,21:07:12,Monday
61248,65074,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
61250,65076,C5E3A0D5,Jalopiina,unknown,industrial,Springfield,20:09:26,Friday
61251,65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [64]:
# Membuat tabel shel_general dari baris df, yang nilai dari kolom 'city'-nya adalah 'Shelbyville'
shel_general= df.loc[df['city']=='Shelbyville']
shel_general

,index,user_id,track,artist,genre,city,time,day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
2,2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
5,5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
9,9,E772D5C0,Pessimist,unknown,dance,Shelbyville,21:20:49,Wednesday
...,...,...,...,...,...,...,...,...
61239,65063,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Shelbyville,21:14:40,Monday
61240,65064,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
61241,65065,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
61242,65066,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday


Selanjutnya akan dibuat fungsi `genre_weekday()` untuk mengetahui jumlah genre yang dimainkan di kedua kota pada hari dan jam yang dijadikan hipotesis

Fungsi tersebut harus menghasilkan informasi tentang 15 genre paling populer pada hari tertentu dalam periode antara dua stempel waktu.

In [65]:
# Membuat fungsi genre_weekday
def genre_weekday(city, day, time1, time2):
    
    genre_df = df[df['city'] == city]

    # pemfilteran berurutan genre_df hanya akan menyimpan baris df yang day-nya sama dengan day
    genre_df = genre_df[genre_df['day'] == day]

    # genre_df hanya akan menyimpan baris df yang time-nya lebih kecil dari time2
    genre_df = genre_df[genre_df['time'] < time2]

    # genre_df hanya akan menyimpan baris df yang time-nya lebih besar dari time1
    genre_df = genre_df[genre_df['time'] > time1] 

    # kelompokkan DataFrame yang telah difilter berdasarkan kolom dengan nama genre, ambil kolom genre, lalu dilakukan count-ing
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()

    # mengurutkan hasilnya dalam urutan menurun
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)

    # objek Series yang menyimpan 15 genre paling populer pada hari tertentu dalam jangka waktu tertentu
    return genre_df_sorted[:15]

Membandingkan hasil dari fungsi `genre_weekday()` untuk Springfield dan Shelbyville pada hari Senin pagi (dari pukul 07.00 hingga 11.00) dan pada hari Jumat malam (dari pukul 17:00 hingga 23:00):

In [66]:
# memanggil fungsi untuk Senin pagi di Springfield
genre_weekday('Springfield', 'Monday', '07:00:00', '11:00:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: genre, dtype: int64

In [67]:
# memanggil fungsi untuk Senin pagi di Shelbyville
genre_weekday('Shelbyville', 'Monday', '07:00:00', '11:00:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: genre, dtype: int64

In [68]:
# memanggil fungsi untuk Jumat malam di Springfield
genre_weekday('Springfield', 'Friday', '17:00:00', '23:00:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [69]:
# memanggil fungsi untuk Jumat malam di Shelbyville
genre_weekday('Shelbyville', 'Friday', '17:00:00', '23:00:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Kesimpulan**

Setelah membandingkan 15 genre teratas pada hari Senin pagi, kita dapat menarik kesimpulan berikut:

1. Pengguna dari Springfield dan Shelbyville mendengarkan musik dengan genre yang sama. Lima genre teratas dari kedua kota sama, hanya genre rock dan elektronik yang bertukar tempat.

2. Di Springfield, jumlah nilai yang hilang ternyata sangat besar, sehingga nilai `'unknown'` pun berada di urutan ke-10. Artinya, nilai-nilai yang hilang mencakup proporsi data yang cukup besar, sehingga fakta ini bisa dijadikan dasar untuk mempertanyakan reliabilitas kesimpulan kami.

Untuk hari Jumat malam, situasinya juga serupa. Genre individual cukup bervariasi, tetapi secara keseluruhan, 15 besar genre untuk kedua kota sama.

Dengan demikian, hipotesis kedua terbukti benar sebagian:
* Pengguna mendengarkan musik yang sama di awal dan di akhir minggu.
* Tidak ada perbedaan yang mencolok antara Springfield dan Shelbyville. Di kedua kota tersebut, pop adalah genre yang paling populer.

Meski demikian, signifikansi jumlah nilai yang hilang membuat hasil ini patut dipertanyakan. Di Springfield, terdapat begitu banyak nilai hilang yang memengaruhi hasil 15 genre teratas kita. Jika kita tidak memiliki nilai-nilai yang hilang tersebut, hasilnya mungkin akan berbeda.

### Hipotesis 3: preferensi genre di kota Springfield dan Shelbyville <a id='genre'></a>

Hipotesis: pendengar di kota Shelbyville menyukai musik rap, sementara pendengar di kota Springfield lebih menyukai pop.

In [70]:
# Membuat series untuk mengetahui total genre yang dimainkan dikota Springfield
spr_genres=spr_general.groupby(['genre'])['track'].count().sort_values(ascending=False)
spr_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
               ... 
metalcore         1
marschmusik       1
malaysian         1
lovers            1
ïîï               1
Name: track, Length: 250, dtype: int64

Tampilkan 10 baris pertama dari `spr_genres`:

In [71]:
# Menampilkan data 10 baris pertama dari spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64

Sekarang, lakukan hal yang sama juga pada data dari kota Shelbyville.

In [72]:
#Membuat series untuk mengetahui total genre yang dimainkan dikota Shelbyville
shel_genres=shel_general.groupby(['genre'])['track'].count().sort_values(ascending=False)
shel_genres

genre
pop           2431
dance         1932
rock          1879
electronic    1736
hiphop         960
              ... 
mandopop         1
leftfield        1
laiko            1
jungle           1
worldbeat        1
Name: track, Length: 202, dtype: int64

Tampilkan 10 baris pertama dari `shel_genres`:

In [73]:
# menampilkan 10 baris pertama dari shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track, dtype: int64

**Kesimpulan**

Hipotesis ini terbukti benar sebagian:
* Musik pop adalah genre yang paling populer di Springfield, seperti yang kita perkirakan.
* Namun, musik pop ternyata sama populernya baik di Springfield maupun di Shelbyville, dan musik rap ternyata tidak masuk ke daftar 5 besar genre untuk kedua kota tersebut.


# Temuan <a id='end'></a>

Kita telah menguji tiga hipotesis berikut:

1. Aktivitas pengguna berbeda-beda tergantung pada hari dan kotanya.
2. Pada hari Senin pagi, penduduk Springfield dan Shelbyville mendengarkan genre yang berbeda. Hal ini juga berlaku untuk hari Jumat malam.
3. Pendengar di kota Springfield dan Shelbyville memiliki preferensi yang berbeda. Baik di Springfield maupun di Shelbyville, pengguna lebih menyukai musik pop.

Setelah menganalisis data yang tersedia, kita dapat menyimpulkan bahwa:

1. Aktivitas pengguna di Springfield dan Shelbyville bergantung pada hari dalam seminggu, meskipun kedua kota ini bervariasi dalam berbagai cara.

Hipotesis pertama dapat diterima sepenuhnya.

2. Preferensi musik tidak bervariasi secara signifikan sepanjang minggu di Springfield dan Shelbyville. Kita bisa melihat perbedaan kecil dalam urutan pada hari Senin, tetapi:
* Baik di Springfield maupun di Shelbyville, pengguna paling banyak mendengarkan musik pop.

Oleh karena itu, hipotesis ini tidak dapat kita terima. Penting juga untuk mengingat bahwa hasil yang diperoleh bisa saja berbeda seandainya kita tidak memiliki nilai yang hilang.

3. Ternyata, preferensi musik pengguna dari kota Springfield dan Shelbyville sangatlah mirip.

Hipotesis ketiga ditolak. Jika memang terdapat perbedaan preferensi, sayangnya hal ini tidak dapat kita ketahui dari data ini.


[Kembali ke Daftar Isi](#back)